In [1]:
# Setup:
import os
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Function for uploading json files into AFS Blob

In [ ]:
def json_upload(jso):
     """
        Uploads the file into the server, checking beforehand for existence of other files related 
        by username, found in the file and giving ID based on the date of opening

        Parameters
        ----------
        jso : string containing path to a .json file
            An object created from opening of a .json file beeing up for upload
    """
    # Retrieve the connection string for use with the application. The storage
    # connection string is stored in an environment variable on the machine
    # running the application called AZURE_STORAGE_CONNECTION_STRING. If the environment variable is
    # created after the application is launched in a console or with Visual Studio,
    # the shell or application needs to be closed and reloaded to take the
    # environment variable into account.
     connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    # Create the BlobServiceClient object
     blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    # Retriving username and first datetime from the file
     with open(jso) as file:
          dict = json.load(file)
          username = dict["username"]
          dt = dict["operations"][0]["date"]
          dt = str.replace(dt, "/", "-")
          dt = str.replace(dt, ":", "%")
          name = "{}/{}.json".format(username, dt)
     with open(jso) as jsf:
          data = jsf.read()
    # Download a list of files in the 'jsons' container
     container_client = blob_service_client.get_container_client(container='jsons')
     bl = container_client.list_blobs()
     filenames = []
     for b in bl:
         filenames.append(b.name)
    # Checking if the file already exists:
     if name in filenames:
          print("File{} already exists".format(name))
     else:
          blob_client = blob_service_client.get_blob_client(container='jsons', blob=name)
          blob_client.upload_blob(data, blob_type="BlockBlob")